In [86]:
import pandas as pd
import numpy as np

In [99]:
# 1. load data
df = pd.read_csv('results_nicolas_all.csv')
orphan = pd.read_excel('2024.10.29. Orphan Designations.BK.xlsx')
approval = pd.read_excel('compilation_of_cder_nme_and_new_biologic_approvals_1985-2023.BK.xlsx')

# 2. define columns to add
orphan_cols_to_add = [
    "Date Designated",
    "Orphan Designation",
    "Orphan Designation Status",
    "Date Designation Withdrawn or Revoked",
    "FDA Orphan Approval Status",
    "Approved Labeled Indication",
    "Marketing Approval Date",
    "Exclusivity End Date",
    "Exclusivity Protected Indication * (Shown for approvals from Jan. 1, 2013, to the present)",
    "Sponsor Company"
]
approval_cols_to_add = [
    "Proprietary Name",
    "Active Ingredient/Moiety",
    "Applicant",
    "NDA/BLA",
    "Application Number(1)",
    "Application Number(2)",
    "FDA Receipt Date",
    "FDA Approval Date",
    "Approval Year",
    "Abbreviated Indication(s)",
    "Approved Use(s)",
    "Review Designation",
    "Orphan Drug Designation",
    "Accelerated Approval",
    "Breakthrough Therapy Designation",
    "Fast Track Designation"
]

# 3. add orphan columns

# 3.1 extract standardized active ingredient
df['row_identifier'] = df['title'].apply(lambda x: x.split(';')[1].lower().strip() if ';' in x else None)
orphan['row_identifier'] = orphan.apply(lambda x: str(x['Trade Name']).lower().strip(),axis=1)

# 3.2 merge dataframes
df = df.merge(
    orphan[['row_identifier']+orphan_cols_to_add],
    how="left",
    on="row_identifier")

# 4. add approval columns

# 4.1 synchronize columns
approval.rename(columns={'Proprietary  Name':'Proprietary Name',
                         ' Application Number(1)':'Application Number(1)',
                         ' Application Number(2)':'Application Number(2)'},inplace=True)

# 4.2 extract standardized application number
approval['row_identifier'] = approval.apply(lambda x: x['Application Number(1)'],axis=1).astype(str)
df['row_identifier'] = df['Application No.'].apply(lambda x: ''.join(filter(str.isdigit, str(x)))).astype(str)

# 4.3 merge dataframes
df = df.merge(
    approval[['row_identifier']+approval_cols_to_add],
    how="left",
    on="row_identifier")

# 5. save dataframe
df.to_csv('results_nicolas_all_compiled.csv',index=False)